# Sleep stage classification: Random Forest & Hidden Markov Model
____

This model aims to classify sleep stages based on two EEG channel. We will use the features extracted in the `pipeline.ipynb` notebook as the input to a Random Forest. The output of this model will then be used as the input of a HMM. We will implement our HMM the same as in this paper (Malafeev et al., « Automatic Human Sleep Stage Scoring Using Deep Neural Networks »).

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

# Ensure parent folder is in PYTHONPATH
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
%matplotlib inline

from itertools import groupby

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import mne

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (StandardScaler)
from sklearn.model_selection import (train_test_split, KFold)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score,
                             confusion_matrix,
                             plot_confusion_matrix,
                             classification_report,
                             f1_score,
                             cohen_kappa_score,
                             log_loss)

from hmmlearn.hmm import MultinomialHMM
from constants import (SLEEP_STAGES_VALUES, N_STAGES)

## Load the features
___

In [ ]:
SUBJECT_IDX = 0
NIGHT_IDX = 1


In [ ]:
X_init = np.load("../data/x_features.npy", allow_pickle=True)
y_init = np.load("../data/y_observations.npy", allow_pickle=True)


In [ ]:
X_init = np.vstack(X_init)
y_init = np.hstack(y_init)
print(X_init.shape)
print(y_init.shape)


In [ ]:
print("Number of subjects: ", np.unique(X_init[:,SUBJECT_IDX]).shape[0]) # Some subject indexes are skipped, thus total number is below 83 (as we can see in https://physionet.org/content/sleep-edfx/1.0.0/)
print("Number of nights: ", len(np.unique([f"{int(x[0])}-{int(x[1])}" for x in X_init[:,SUBJECT_IDX:NIGHT_IDX+1]])))


## Downsizing sets
___

We will use the same set for all experiments. It includes the first 20 subjects, and excludes the 13th, because it only has one night.

The last subject will be put in the test set. 

In [ ]:
# Filtering to only keep first 20 subjects

X_20 = X_init[np.isin(X_init[:,SUBJECT_IDX], range(20))]
y_20 = y_init[np.isin(X_init[:,SUBJECT_IDX], range(20))]

print(X_20.shape)
print(y_20.shape)

In [ ]:
# Exclude the subject with only one night recording (13th)

MISSING_NIGHT_SUBJECT = 13

X = X_20[X_20[:,SUBJECT_IDX] != MISSING_NIGHT_SUBJECT]
y = y_20[X_20[:,SUBJECT_IDX] != MISSING_NIGHT_SUBJECT]

print(X.shape)
print(y.shape)

In [ ]:
print("Number of subjects: ", np.unique(X[:,SUBJECT_IDX]).shape[0]) # Some subject indexes are skipped, thus total number is below 83 (as we can see in https://physionet.org/content/sleep-edfx/1.0.0/)
print("Subjects available: ", np.unique(X[:,SUBJECT_IDX]))
print("Number of nights: ", len(np.unique([f"{int(x[0])}-{int(x[1])}" for x in X[:,SUBJECT_IDX:NIGHT_IDX+1]])))

## Train, validation and test sets
___

The two nights recording of the last subject (no 19) will be the test set. The rest will be the train and validation sets. 

In [ ]:
def train_test_split_(X, subject_test=19):
    test_indexes = np.where(X[:,SUBJECT_IDX] == subject_test)[0]
    train_indexes = list(set(range(X.shape[0])) - set(test_indexes))

    assert X.shape[0] == len(train_indexes)+len(test_indexes), "Total train and test sets must corresponds to all dataset"
    
    X_test = X[test_indexes,:]
    y_test = y[test_indexes]
    X_train = X[train_indexes,:]
    y_train = y[train_indexes]
    
    return X_test, X_train, y_test, y_train
    
X_test, X_train_valid, y_test, y_train_valid = train_test_split_(X)
print(X_test.shape, X_train_valid.shape, y_test.shape, y_train_valid.shape)

## Random forest validation
___

In [ ]:
NB_KFOLDS = 5
NB_CATEGORICAL_FEATURES = 2
NB_FEATURES = 48

def get_random_forest_model():
    return Pipeline([
        ('scaling', ColumnTransformer([
            ('pass-through-categorical', 'passthrough', list(range(NB_CATEGORICAL_FEATURES))),
            ('scaling-continuous', StandardScaler(copy=False), list(range(NB_CATEGORICAL_FEATURES,NB_FEATURES)))
        ])),
        ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1))
    ])

In [ ]:
%%time

accuracies = []
f1_scores = []
emission_matrix = np.zeros((N_STAGES,N_STAGES))

for train_index, valid_index in KFold(n_splits=5).split(X_train_valid):
    print("TRAIN:", train_index, "VALID:", valid_index)

    # We drop the subject and night indexes
    X_train, X_valid = X_train_valid[train_index, 2:], X_train_valid[valid_index, 2:]
    y_train, y_valid = y_train_valid[train_index], y_train_valid[valid_index]
    
    # Scaling features and model training
    training_pipeline = get_random_forest_model()
    training_pipeline.fit(X_train, y_train)
    
    # Validation
    y_valid_pred = training_pipeline.predict(X_valid)

    print("------ STANDARD SCALER --------------")
    accuracies.append(round(accuracy_score(y_valid, y_valid_pred),2))
    f1_scores.append(f1_score(y_valid, y_valid_pred, average="micro"))
    print(confusion_matrix(y_valid, y_valid_pred))
    print(classification_report(y_valid, y_valid_pred, target_names=SLEEP_STAGES_VALUES.keys()))
    print("Agreement score (Cohen Kappa): ", cohen_kappa_score(y_valid, y_valid_pred))
    
    for y_pred, y_true in zip(y_valid_pred, y_valid):
        emission_matrix[y_true, y_pred] += 1
    

emission_matrix = emission_matrix / emission_matrix.sum(axis=1, keepdims=True)

print(f"\n\nAccuracies accross {NB_KFOLDS} folds: {accuracies}")
print(f"Mean F1-score: {np.mean(f1_scores):0.2f}")

## Random forest training and testing
___

In [ ]:
testing_pipeline = get_random_forest_model()

testing_pipeline.fit(X_train_valid[:, 2:], y_train_valid);
y_train_valid_pred = testing_pipeline.predict(X_train_valid[:, 2:])

In [ ]:
y_test_pred = testing_pipeline.predict(X_test[:,2:])

print(confusion_matrix(y_test, y_test_pred))

print(classification_report(y_test, y_test_pred, target_names=SLEEP_STAGES_VALUES.keys()))

print("Agreement score (Cohen Kappa): ", cohen_kappa_score(y_test, y_test_pred))

## Hidden Model Markov
___

In [ ]:
def compute_hmm_matrices(y, subject_night):
    transition_matrix = np.zeros((N_STAGES,N_STAGES))
    start_matrix = np.zeros((N_STAGES))

    for night in groupby(zip(y, subject_night), key=lambda x: f"subject{int(x[1][0])}-night{int(x[1][1])}"):
        print(f"Computing file: {night[0]}")
        current_y = np.array([x[0] for x in night[1]])
        start_matrix[current_y[0]] += 1

        for transition in zip(current_y[:-1], current_y[1:]):
            transition_matrix[transition[0], transition[1]] += 1
            
    transition_matrix = transition_matrix/transition_matrix.sum(axis=1, keepdims=True)
    start_matrix = start_matrix/start_matrix.sum()
    
    return transition_matrix, start_matrix
    
transition_matrix, start_matrix = compute_hmm_matrices(y_train_valid, X_train_valid[:,0:2])

In [ ]:
hmm_model = MultinomialHMM(n_components=N_STAGES)

hmm_model.transmat_ = transition_matrix
hmm_model.startprob_ = start_matrix
hmm_model.emissionprob_ = emission_matrix

In [ ]:
y_hmm_pred = hmm_model.predict(y_test_pred.reshape(-1, 1))

In [ ]:
print(confusion_matrix(y_test, y_hmm_pred))

print(classification_report(y_test, y_hmm_pred, target_names=SLEEP_STAGES_VALUES.keys()))

print("Agreement score (Cohen Kappa): ", cohen_kappa_score(y_test, y_hmm_pred))

In [ ]:
def print_hypnogram(y, label):
    y_hypno = y

    # Tweak to make REM between N1 and W
    y_hypno[y_hypno == SLEEP_STAGES_VALUES["REM"]] = -1    
    y_hypno[y_hypno == SLEEP_STAGES_VALUES["N3"]] = SLEEP_STAGES_VALUES["REM"]
    y_hypno[y_hypno == SLEEP_STAGES_VALUES["N2"]] = SLEEP_STAGES_VALUES["N3"]
    y_hypno[y_hypno == SLEEP_STAGES_VALUES["N1"]] = SLEEP_STAGES_VALUES["N2"]
    y_hypno[y_hypno == -1] = SLEEP_STAGES_VALUES["N1"]
    # -------------------------------- #
    
    y_hypno = np.array([(index*30, stage) for index, stage in enumerate(y_hypno)])    
    
    plt.plot([y[0]/3600 for y in y_hypno], [y[1] for y in y_hypno], label=label)
    plt.xlabel("Time since bed time (hours)")
    plt.ylabel("Sleep stage")
    plt.yticks(range(5), ['W','REM','N1','N2','N3'])
    plt.legend()
    

In [ ]:
plt.rcParams["figure.figsize"] = (20,5)

print("Test subjects are subjects: ", np.unique(X_test[:,0]))
print("BEFORE HMM")
for test_subject in np.unique(X_test[:,0]):
    test_subject_indexes = [idx for idx, elem in enumerate(X_test) if elem[0] == test_subject]
    
    for night_idx in np.unique(X_test[test_subject_indexes,1]):
        test_night_subject_indexes = [
            idx for idx, elem in enumerate(X_test)
            if elem[0] == test_subject and elem[1] == night_idx]
        print_hypnogram(y_test[test_night_subject_indexes], label="scored")

        print_hypnogram(y_test_pred[test_night_subject_indexes], label="predicted")


        plt.title(f"Hypnogram of subject {test_subject}, night {night_idx}")
        plt.grid(axis='y')
        plt.gca().invert_yaxis()
        plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (20,5)

print("Test subjects are subjects: ", np.unique(X_test[:,0]))
print("AFTER HMM")

for test_subject in np.unique(X_test[:,0]):
    test_subject_indexes = [idx for idx, elem in enumerate(X_test) if elem[0] == test_subject]
    
    for night_idx in np.unique(X_test[test_subject_indexes,1]):
        test_night_subject_indexes = [
            idx for idx, elem in enumerate(X_test)
            if elem[0] == test_subject and elem[1] == night_idx]
        print_hypnogram(y_test[test_night_subject_indexes], label="scored")

        print_hypnogram(y_hmm_pred[test_night_subject_indexes], label="predicted")


        plt.title(f"Hypnogram of subject {test_subject}, night {night_idx}")
        plt.grid(axis='y')
        plt.gca().invert_yaxis()
        plt.show()